# Save and load the model

이 섹션에서 우리는 모델의 상태를 어떻게 저장할 것인가 살펴볼 것이다,  

- saving
- loading
- running model predictions

In [1]:
import torch
import torch.onnx as onnx
import torchvision.models as models

## Saving and Loading Model Weights

PyTorch 모델들은 학습된 파라미터를 내부 상태 사전 자료형에 저장한다,  
이는 `state_dict`로 불린다.  
이는 `torch.save()` 메소드를 통해 저장할 수 있다.

In [2]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/livlikwav/.cache/torch/hub/checkpoints/vgg16-397923af.pth
4.4%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

10.3%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

15.9%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.


model weights를 불러오기 위해서, 우리는 같은 모델 인스턴스를 먼저 생성할 필요가 있다,  
그리고 파라미터들을 `load_state_dict()` 메소드를 사용해서 불러와야한다.

In [3]:
model = models.vgg16() # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### NOTE

inferencing(추론) 하기 전에 `model.eval()`을 꼭 호출해야한다.  
이를 통해 dropout과 batch normalization layers를 evalution mode로 설정한다.  

이거 호출안하면 일관되지 않는 inference 결과값들 나올꺼임

## Saving and Loading Models with Shapes

model weights를 불러올 때,  
우리는 모델 클래스를 먼저 초기화할 필요가 있다,  
왜냐하면 그 클래스가 신경망의 구조를 정의하기 때문이다.

우리는 모델의 구조까지 함께 저장하고 싶을 수 있다,  
이 경우에 우리는 `model`을 저장하는 함수에 전달한다. (`model.state_dict()` 말고 `model` 그대로를)

In [4]:
torch.save(model, 'model.pth')

우리는 이 모델을 이렇게 불러올 수 있다.

In [5]:
model = torch.load('model.pth')

### NOTE

이 접근 방법은 모델을 serializing(직렬화)할 때 Python의 pickle 모듈을 사용한다,  
그러므로 모델을 불러올 때 실제 클래스 정의에 의존한다.

(내 이해)불러온 인스턴스를 인터프리터에서 이해하려면, 해당 클래스가 먼저 정의되어 있어야 하나보다.  
근데 그러면 결국 원본 소스코드도 같이 저장해야하네?  

위에 예시는 dict로 저장해서 용량 적고, 모델 인스턴스 직접 생성해서 거기다가 붓는 느낌이면,  
이건 인스턴스 그대로 떠서 저장하는데, 결국 클래스 정의는 있어야 하는 느낌

## Exporting model to ONNX

파이토치는 native하게 ONNX 내보내기도 지원한다.  
그러나, 파이토치의 실행 그래프의 동적인 구성으로, 내보내기 과정에서 실행 그래프를 순회할 필요가 있다.  
이러한 이유로, 적절한 사이즈의 테스트 변수가 내보내기 함수에 전달되어야 한다.  
(우리 예시에서는 알맞은 사이즈의 더비 0값 텐서를 생성한다)

In [6]:
input_image = torch.zeros((1, 3, 224, 224))
onnx.export(model, input_image, 'model.onnx')

ONNX 모델로 할 수 있는 것들이 많다.

- 다른 플랫폼에서 인퍼런스 수행하기
- 다른 프로그래밍 언어에서 인퍼런스 수행하기

더 자세한 내용은 ONNX tutorial을 참고해라

## Congratulations!

파이토치 초심자 튜토리얼을 완료했다!